In [ ]:
# Install necessary libraries
!pip install pandas numpy re nltk scikit-learn tensorflow

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'imdb-dataset-of-50k-movie-reviews' dataset.
Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews


In [ ]:
!pip install tensorflow --ignore-installed


  Using cached tensorflow-2.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached protobuf-6.32.1-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached termcolor-3.1.0-py3-none-

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Attention, LayerNormalization, Dropout


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [ ]:
# Download NLTK data (if not already downloaded)
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")


In [ ]:
df['cleaned_review'] = df['review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
df['cleaned_review'] = df['cleaned_review'].apply(lambda x: x.lower())
stop_words = set(stopwords.words('english'))
df['cleaned_review'] = df['cleaned_review'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['sentiment'], test_size=0.2, random_state=42)


In [ ]:
# Label Encoding for Sentiment Column
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token="")
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_train_padded = pad_sequences(X_train_sequences, maxlen=100, truncating='post', padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=100, truncating='post', padding='post')



In [ ]:
# Transformer Model Building
embedding_dim = 32  # We can Adjust the embedding dimension based on our preference
num_heads = 2
ff_dim = 32

inputs = Input(shape=(100,))
embedding_layer = Embedding(input_dim=10000, output_dim=embedding_dim)(inputs)
transformer_block = embedding_layer

for _ in range(num_heads):
    attn_output = Attention(use_scale=True)([transformer_block, transformer_block])
    transformer_block = LayerNormalization(epsilon=1e-6)(transformer_block + attn_output)
    transformer_block = Dropout(0.1)(transformer_block)

transformer_block = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(transformer_block)
transformer_block = tf.keras.layers.GlobalAveragePooling1D()(transformer_block)
transformer_block = Dropout(0.1)(transformer_block)
transformer_block = Dense(20, activation='relu')(transformer_block)
output_layer = Dense(1, activation='sigmoid')(transformer_block)

model = Model(inputs=inputs, outputs=output_layer)


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_padded, y_train_encoded, epochs=5, validation_data=(X_test_padded, y_test_encoded))


Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.8204 - loss: 0.3934 - val_accuracy: 0.8666 - val_loss: 0.3129
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.8876 - loss: 0.2703 - val_accuracy: 0.8668 - val_loss: 0.3124
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.9057 - loss: 0.2326 - val_accuracy: 0.8618 - val_loss: 0.3269
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.9161 - loss: 0.2042 - val_accuracy: 0.8602 - val_loss: 0.3780
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.9277 - loss: 0.1739 - val_accuracy: 0.8595 - val_loss: 0.3861


In [ ]:
model.save('/content/transformer_sentiment_model.h5')


In [ ]:
y_pred_prob = model.predict(X_test_padded)
y_pred = (y_pred_prob > 0.5).astype(int)

# Converting back to original labels
y_pred_original = label_encoder.inverse_transform(y_pred.flatten())

print("Accuracy:", accuracy_score(y_test_encoded, y_pred))
print("Classification Report:\n", classification_report(y_test_encoded, y_pred))


220/313 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

In [ ]:
import csv

def predict_sentiment_transformer(review):
    cleaned_review = re.sub(r'[^a-zA-Z\s]', '', review.lower())
    cleaned_review = ' '.join(word for word in cleaned_review.split() if word not in stop_words)
    sequence = tokenizer.texts_to_sequences([cleaned_review])
    padded_sequence = pad_sequences(sequence, maxlen=100, truncating='post', padding='post')
    prediction = model.predict(padded_sequence)
    if 0.4 <= prediction <= 0.55:
        return "Neutral"
    elif prediction > 0.55:
        return "Positive"
    else:
        return "Negative"

# Enter your review to get the prediction of the sentiment
with open('/content/revis.csv', 'r') as f:
  reader = csv.reader(f)

  # Iterate over each row
  for row in reader:
    prediction = predict_sentiment_transformer(row[1])
    print(f"First 10 words of the review: {row[1][0:10]}")
    print(f"Predicted Sentiment: {prediction}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
First 10 words of the review: Comentario
Predicted Sentiment: Neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
First 10 words of the review: I feel com
Predicted Sentiment: Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
First 10 words of the review: If only I'
Predicted Sentiment: Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
First 10 words of the review: Platform i
Predicted Sentiment: Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
First 10 words of the review: Avoid at a
Predicted Sentiment: Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
First 10 words of the review: i been ver
Predicted Sentiment: Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
First 10 words of the review: I sent a q
Predicted Sentiment: Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
First 10 words of the review: Just lost 
Predicted Sentiment: Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
First 10 words of the review: Was part o
Predicted Sentiment: Negative
1/1 ━━━━━━━━━━━━━━━━